Let's create `macrograd` from scratch following Andrej Karpathy's guide

In [1]:
# This is all we need to create macrograd
# Amazing, isn't it?
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Using `graphviz` to visualize the computation graph
import graphviz

In [3]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [41]:
class Value:
    
    def __init__(self, data, _op=None, _prev=set(), label=""):
        self.data = data
        self._op = _op
        self._prev = _prev
        self.label = label
        
    def __repr__(self):
        return f"Value(data={self.data}, label={self.label})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        ret = Value(self.data + other.data, _op='+', _prev={self, other})
        return ret

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        ret = Value(self.data * other.data, _op='*', _prev={self, other})
        return ret
    
    def __radd__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        ret = Value(self.data + other.data, _op='+', _prev={self, other})
        return ret
    
    def __rmul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        ret = Value(self.data * other.data, _op='*', _prev={self, other})
        return ret

In [42]:
a = Value(3, label="a")
b = Value(4, label="b")
c = a + b; c.label = 'c'

In [43]:
c.data, c._op, c._prev, c.label

(7, '+', {Value(data=3, label=a), Value(data=4, label=b)}, 'c')